# Running Dataflow jobs from Notebook

### Running python file from Notebook

In [14]:
!python dataflow-job.py --runner DataflowRunner --project crazy-hippo-01 --temp_location gs://crazy-hippo-01/dataflow_beam_data --staging_location gs://crazy-hippo-01/dataflow_beam_data --region us-central1

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1653: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:901: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


#### Code from file:

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import StandardOptions, PipelineOptions

options = PipelineOptions()
options.view_as(StandardOptions).streaming = False

#Transformation Function #1 
def selectData(element) :
      return element.split(',')

#Transformation Function #2
def filtering(record):
      return record[3] == 'Accounts'

pipeline1 = beam.Pipeline(options=options)

#BigQuery Parameters
table_spec = 'crazy-hippo-01:department_dataflow.group_by_name'
table_schema = 'name:STRING, count:INTEGER'

#Pipeline
dep_data_count = (
    pipeline1
      |'Read from file' >> beam.io.ReadFromText('gs://crazy-hippo-01/dataflow_beam_data/dept-data.txt')
      |'Select_data' >> beam.Map(selectData)
      |'Filter record on Accounts' >> beam.Filter(filtering)
      |'Create Dict of Records' >> beam.Map(lambda record : (record[1], 1))
      |'Apply CombinePerKey on Records' >> beam.CombinePerKey(sum)
      |'Make into Dict' >> beam.Map(lambda x: {"name": x[0], "count": x[1]})
      #|'Write to Cloud Storage' >> beam.io.WriteToText('gs://crazy-hippo-01/dataflow_beam_data/output_new')
      |'Write to BQ' >> beam.io.WriteToBigQuery(
                                                table_spec,
                                                schema=table_schema,
                                                write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                                                create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
)

#Execution
pipeline1.run().wait_until_finish()